<a href="https://colab.research.google.com/github/mmsiqueira98/Campeonato-Brasileiro/blob/main/ESPN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#WEB SCRAPING ESPN PARA DADOS ESTATÍSTICOS FUTEBOL

O objetivo é fazer o scraping do site ESPN com os dados dos times de futebol, para

##Pacotes Bibliotecas e links

In [1]:
pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 5.8 MB/s eta 0:00:00


In [2]:
import requests
from bs4 import BeautifulSoup
import csv
from requests import HTTPError
from xml.dom import minidom
import pandas as pd
from unidecode import unidecode
import matplotlib.pyplot as plt
import re
from typing import List
from bs4.element import Tag

#Scrapping

Como observei na estrutura do site da ESPN, os times são categorizados pela seguinte estrutura: 

a = **https://www.espn.com/soccer/team/_/id/6273/gremio**

b = **https://www.espn.com/soccer/team/_/id/6273**

  o valor que vem depois de ID é o que eu preciso, na realidade a descrição do time, como nesse exemplo está gremio, pouco me importa. já que se eu colocar o link até o valor de 6273, serei entregue a pagina responsável pelo grêmio.
Logo, a = b

A primeira coisa que preciso fazer é definir essa função **crawl_website** que será responsável por fazer o scrapping dos sites que eu for utilizar futuramente

In [3]:
def crawl_website(url: str, archive: str) -> str: 
  try: 
    resposta = requests.get(url)
    
  
  except HTTPError as exc: 
    print(exc)
  
  else: 
    b = resposta.text
    soup = BeautifulSoup(b, 'html.parser')
    with open(f'{archive}.html', 'w') as file:
      file.write(soup.prettify())
  
  pagina = BeautifulSoup(open(f'{archive}.html',mode='r'),'html.parser')
  return pagina

In [6]:
def find_all(elementos: List[Tag], letra: str, classe: str, objeto: str):
  elementos = elementos.find_all(letra, class_=classe)
  lista = []
  for elemento in elementos: 
    valor_href = elemento[objeto]
    lista.append(valor_href)

  return lista

Agora, a URL, onde estão as informações dos times: 

**https://www.espn.com/soccer/teams/_/league/bra.1**

In [7]:
times = 'https://www.espn.com/soccer/teams/_/league/bra.1'
id_times = crawl_website(url = times, archive = 'BrasileiraoSerieA')
lista = find_all(elementos = id_times, letra = 'a', classe = 'AnchorLink', objeto = 'href' )
print(lista)

['/', '/nfl/', '/nba/', '/nhl/', '/mlb/', '/soccer/', '/college-sports/', 'http://www.espn.com', 'https://www.espn.com/espnplus/?om-navmethod=topnav', 'http://www.espn.com/watch/', 'http://www.espn.com/espnradio/index', '/fantasy/', 'http://www.espn.com', '/soccer/', '/soccer/', '/soccer/scoreboard', '/soccer/schedule', '/soccer/transfers-news-and-features/', '/soccer/teams', '/soccer/competitions', '/soccer/table/_/league/eng.1', '/soccer/team/_/id/660/united-states', '/soccer/team/_/id/2765/united-states', '#', '/soccer/team/_/id/6154/america-mg', '/soccer/team/_/id/6154/america-mg', '/soccer/team/fixtures/_/id/6154/america-mg', '/soccer/team/stats/_/id/6154/america-mg', '/soccer/team/_/id/3458/athletico-pr', '/soccer/team/_/id/3458/athletico-pr', '/soccer/team/fixtures/_/id/3458/athletico-pr', '/soccer/team/stats/_/id/3458/athletico-pr', '/soccer/team/_/id/7632/atletico-mg', '/soccer/team/_/id/7632/atletico-mg', '/soccer/team/fixtures/_/id/7632/atletico-mg', '/soccer/team/stats/_/id

In [8]:
elementos = id_times.find_all('a', class_='AnchorLink')
lista = []
for elemento in elementos: 
  valor_href = elemento['href']
  lista.append(valor_href)

print(lista)

['/', '/nfl/', '/nba/', '/nhl/', '/mlb/', '/soccer/', '/college-sports/', 'http://www.espn.com', 'https://www.espn.com/espnplus/?om-navmethod=topnav', 'http://www.espn.com/watch/', 'http://www.espn.com/espnradio/index', '/fantasy/', 'http://www.espn.com', '/soccer/', '/soccer/', '/soccer/scoreboard', '/soccer/schedule', '/soccer/transfers-news-and-features/', '/soccer/teams', '/soccer/competitions', '/soccer/table/_/league/eng.1', '/soccer/team/_/id/660/united-states', '/soccer/team/_/id/2765/united-states', '#', '/soccer/team/_/id/6154/america-mg', '/soccer/team/_/id/6154/america-mg', '/soccer/team/fixtures/_/id/6154/america-mg', '/soccer/team/stats/_/id/6154/america-mg', '/soccer/team/_/id/3458/athletico-pr', '/soccer/team/_/id/3458/athletico-pr', '/soccer/team/fixtures/_/id/3458/athletico-pr', '/soccer/team/stats/_/id/3458/athletico-pr', '/soccer/team/_/id/7632/atletico-mg', '/soccer/team/_/id/7632/atletico-mg', '/soccer/team/fixtures/_/id/7632/atletico-mg', '/soccer/team/stats/_/id

Agora, eu preciso separar o soccer/team, excluir isso e juntar o resultado em uma lista, sem que haja repetições.

In [9]:
padrao = re.compile(r'/soccer/team/stats/_/id/(.*)')
valores_filtrados = []

for valor in lista: 
  resultado = padrao.search(valor)
  if resultado: 
    informacoes = resultado.group(1)
    valores_filtrados.append(informacoes)
  
id_partida = list(set(valores_filtrados))
times = [valor.split('/') for valor in valores_filtrados]

time = []
id = []

for i in times: 
  time.append(i[1])
  id.append(i[0])

print(time)
print(id)

['america-mg', 'athletico-pr', 'atletico-mg', 'bahia', 'botafogo', 'corinthians', 'coritiba', 'cruzeiro', 'cuiaba', 'flamengo', 'fluminense', 'fortaleza', 'goias', 'gremio', 'internacional', 'palmeiras', 'red-bull-bragantino', 'santos', 'sao-paulo', 'vasco-da-gama']
['6154', '3458', '7632', '9967', '6086', '874', '3456', '2022', '17313', '819', '3445', '6272', '3395', '6273', '1936', '2029', '6079', '2674', '2026', '3454']


In [10]:
year = ['2017', '2018', '2019', '2020', '2021', '2022']
time = []
id = []

for i in times: 
  time.append(i[1])
  id.append(i[0])

print(time)
print(id)

['america-mg', 'athletico-pr', 'atletico-mg', 'bahia', 'botafogo', 'corinthians', 'coritiba', 'cruzeiro', 'cuiaba', 'flamengo', 'fluminense', 'fortaleza', 'goias', 'gremio', 'internacional', 'palmeiras', 'red-bull-bragantino', 'santos', 'sao-paulo', 'vasco-da-gama']
['6154', '3458', '7632', '9967', '6086', '874', '3456', '2022', '17313', '819', '3445', '6272', '3395', '6273', '1936', '2029', '6079', '2674', '2026', '3454']


Ótimo, esse resultado é tudo o que eu preciso para esse momento.

Agora vamos criar a variável link.

In [11]:
link = []

for i in id: 
  for j in year: 
    a = f'https://www.espn.com/soccer/team/results/_/id/{i}/season/{j}'
    link.append(a)

print(link)

['https://www.espn.com/soccer/team/results/_/id/6154/season/2017', 'https://www.espn.com/soccer/team/results/_/id/6154/season/2018', 'https://www.espn.com/soccer/team/results/_/id/6154/season/2019', 'https://www.espn.com/soccer/team/results/_/id/6154/season/2020', 'https://www.espn.com/soccer/team/results/_/id/6154/season/2021', 'https://www.espn.com/soccer/team/results/_/id/6154/season/2022', 'https://www.espn.com/soccer/team/results/_/id/3458/season/2017', 'https://www.espn.com/soccer/team/results/_/id/3458/season/2018', 'https://www.espn.com/soccer/team/results/_/id/3458/season/2019', 'https://www.espn.com/soccer/team/results/_/id/3458/season/2020', 'https://www.espn.com/soccer/team/results/_/id/3458/season/2021', 'https://www.espn.com/soccer/team/results/_/id/3458/season/2022', 'https://www.espn.com/soccer/team/results/_/id/7632/season/2017', 'https://www.espn.com/soccer/team/results/_/id/7632/season/2018', 'https://www.espn.com/soccer/team/results/_/id/7632/season/2019', 'https://

Por qual motivo eu fiz isso? 

Bom, com o id do time que está no brasileirão da série A, agora eu posso ir até esse domínio do site: 

**https://www.espn.com/soccer/team/results/_/id/{id}/season/{year}**

onde

**id = id do time**

e 

**year = ano do time**

O objetivo é rastrear os jogos de cada time de 2017 até 2022. Mas primeiro, iremos realizar o crawling de todos os dados do site que estão correspondendo aos valores relativos a link.



In [12]:
id_times = []
a = 0
for i in link: 
  a+=1
  partidas = crawl_website(url = i, archive = f'{a}')

Feito isso, Temos agora todos os scraping dos sites com os times nos torneios de 2017 até o momento, considerando apenas aqueles que estão na Série A nesse campeonato. Nesses respectivos crawlings realizados nos temos uma informação muito importante conforme está mostrada nesse link: 

'https://www.espn.com/soccer/matchstats/_/gameId/{informação_desejada}'

Essa informação é o GameId, onde contém o resultado de uma partida de futebol entre dois times, o seu resultado e também, em algumas partidas, as estatísticas durante as partidas de futebol, que é o nosso grande objetivo com esse trabalho.

##Realizando o Web Scrapping dos dados obtidos

Como foi observado anteriormente, os dados foram enumerados de 0 a n, onde 0 é o primeiro crawling e n é o último. Para sabermos qual é o número de crawling's feitos precisamos apenas ver a quantidade links que tem. Sabendo que são 120 então nos temos 0.html a 119.html. Precisamos realizar o scrapping desses dados temo como alvo: 

 /soccer/matchstats/_/gameId/(Informação_desejada)

In [25]:
partidas = []
for a in range(1, 120): 
  pagina = BeautifulSoup(open(f'/content/{a}.html',mode='r'),'html.parser')

  b = find_all(elementos=pagina, letra='a', classe = 'AnchorLink', objeto = 'href')
  padrao = re.compile(r'/gameId/(\d+)')
  valores_filtrados = []

  for valor in b: 
    resultado = padrao.search(valor)
    if resultado: 
      informacoes = resultado.group(1)
      valores_filtrados.append(informacoes)
      partidas.append(informacoes)


14154


Vamos avaliar a quantidade de dados.

In [27]:
partidas_unicas = list(set(partidas))
print(len(partidas_unicas)) 
print(partidas_unicas)

5255
['538297', '511455', '538397', '560148', '538230', '529345', '569775', '631074', '537645', '508278', '528603', '537847', '598754', '529220', '570420', '630999', '538336', '622391', '598536', '530672', '538365', '538114', '598424', '507272', '468964', '476072', '537589', '598569', '568621', '538027', '627227', '631406', '475306', '475953', '631185', '537713', '476122', '530752', '491845', '631368', '475325', '475268', '569875', '598404', '557725', '631320', '598511', '507547', '598597', '598650', '601713', '569910', '631032', '507476', '538021', '631122', '596371', '539189', '538190', '503447', '538319', '560155', '631055', '633495', '532979', '469441', '507781', '469395', '506987', '569689', '476150', '633622', '587494', '569715', '600905', '503448', '470325', '540076', '631037', '506998', '622203', '470381', '600823', '498999', '506958', '592313', '476170', '492110', '476204', '538097', '478705', '508275', '475451', '622400', '598281', '539334', '569624', '529368', '622647', '530

Vamos salvar esses arquivos em csv para não precisar realizar o scrapping novamente até aqui.

In [31]:
nome_do_arquivo = 'id_das_partidas.csv'

with open (nome_do_arquivo, 'w', newline='') as arquivo: 
  escritor_csv = csv.writer(arquivo, delimiter=',')
  escritor_csv.writerow(['Valores'])

  for valor in partidas_unicas: 
    escritor_csv.writerow([valor])

Aqui eu vou simplesmente criar uma lista para prosseguir com o código, para que sua leitura seja menos confusa

o que eu tenho em link? Em Link eu tenho todos os times que estão na Série A desse ano (20) e sua trajetória desde 2017. Logo, eu tenho 6 anos e 20 times o que é igual a 120.

##Próximos Passos

Os próximos passos agora são abrir os sites e filtar aqueles que contém as estatísticas dos times que se confrontaram para que aí sim possamos criar um novo csv com todos esses valores e partir para a Análise de Dados.